# Module 18: Tool Calling

**Goal:** Learn how to give LLMs the ability to call functions and interact with external systems.

**Prerequisites:** Module 17 (LLM Fundamentals)

**Expected Runtime:** ~25 minutes

**Outputs:**
- Designed tool schemas
- Implemented tool execution logic
- Built a simple agent with tools

---

## Setup

In [ ]:
import json
import re
from typing import Dict, Any, Callable, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

## Part 1: Defining Tool Schemas

Tools need clear schemas so the LLM knows what they do and what parameters they need.

In [ ]:
# Define tool schemas (OpenAI function calling format)
TOOL_SCHEMAS = [
    {
        "name": "get_order_status",
        "description": "Get the current status of a customer order. Use when user asks about order status, shipping, or tracking.",
        "parameters": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The order ID (e.g., 'ORD-12345')"
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "search_knowledge_base",
        "description": "Search the help documentation for articles. Use when user asks how to do something or needs help.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search query"
                },
                "limit": {
                    "type": "integer",
                    "description": "Maximum number of results (default: 3)",
                    "default": 3
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "calculate_shipping",
        "description": "Calculate shipping cost for a package. Use when user asks about shipping costs or delivery options.",
        "parameters": {
            "type": "object",
            "properties": {
                "weight_kg": {
                    "type": "number",
                    "description": "Package weight in kilograms"
                },
                "destination": {
                    "type": "string",
                    "description": "Destination city or country"
                },
                "express": {
                    "type": "boolean",
                    "description": "Whether to use express shipping",
                    "default": False
                }
            },
            "required": ["weight_kg", "destination"]
        }
    }
]

print("=== Tool Schemas ===")
for schema in TOOL_SCHEMAS:
    print(f"\n{schema['name']}:")
    print(f"  {schema['description'][:60]}...")
    print(f"  Parameters: {list(schema['parameters']['properties'].keys())}")

## Part 2: Implementing Tool Functions

In [ ]:
# Mock database
ORDERS_DB = {
    "ORD-12345": {"status": "shipped", "tracking": "1Z999AA10123456784", "eta": "2024-01-18"},
    "ORD-67890": {"status": "processing", "tracking": None, "eta": "2024-01-20"},
    "ORD-11111": {"status": "delivered", "tracking": "1Z999AA10123456785", "eta": None},
}

KNOWLEDGE_BASE = [
    {"title": "Password Reset", "content": "Click 'Forgot Password' on login page. Check email for reset link."},
    {"title": "Return Policy", "content": "Returns accepted within 30 days. Items must be unused."},
    {"title": "Shipping Info", "content": "Standard: 5-7 days. Express: 2-3 days. Free over $50."},
    {"title": "Account Security", "content": "Enable 2FA in Settings > Security. Use strong passwords."},
]

def get_order_status(order_id: str) -> Dict[str, Any]:
    """Get the status of an order."""
    if order_id not in ORDERS_DB:
        return {"error": f"Order {order_id} not found"}
    
    order = ORDERS_DB[order_id]
    return {
        "order_id": order_id,
        "status": order["status"],
        "tracking_number": order["tracking"],
        "estimated_delivery": order["eta"]
    }

def search_knowledge_base(query: str, limit: int = 3) -> Dict[str, Any]:
    """Search the knowledge base."""
    query_lower = query.lower()
    results = []
    
    for article in KNOWLEDGE_BASE:
        if query_lower in article["title"].lower() or query_lower in article["content"].lower():
            results.append(article)
    
    return {
        "query": query,
        "num_results": len(results[:limit]),
        "articles": results[:limit]
    }

def calculate_shipping(weight_kg: float, destination: str, express: bool = False) -> Dict[str, Any]:
    """Calculate shipping cost."""
    base_rate = 5.99
    cost = base_rate * weight_kg
    days = 7
    
    if express:
        cost *= 2
        days = 2
    
    return {
        "weight_kg": weight_kg,
        "destination": destination,
        "shipping_type": "express" if express else "standard",
        "cost_usd": round(cost, 2),
        "estimated_days": days
    }

# Tool registry
TOOLS = {
    "get_order_status": get_order_status,
    "search_knowledge_base": search_knowledge_base,
    "calculate_shipping": calculate_shipping,
}

# Test tools
print("=== Testing Tools ===")
print("\nget_order_status('ORD-12345'):")
print(json.dumps(get_order_status('ORD-12345'), indent=2))

print("\nsearch_knowledge_base('password'):")
print(json.dumps(search_knowledge_base('password'), indent=2))

## Part 3: Simulating LLM Tool Selection

In [ ]:
def simulate_llm_tool_selection(query: str) -> Optional[Dict[str, Any]]:
    """
    Simulate LLM selecting a tool based on query.
    In production, this would be the LLM's function_call response.
    """
    query_lower = query.lower()
    
    # Order-related queries
    if any(word in query_lower for word in ['order', 'tracking', 'shipped', 'delivery']):
        # Extract order ID
        match = re.search(r'ORD-\d+', query, re.IGNORECASE)
        if match:
            return {
                "tool": "get_order_status",
                "arguments": {"order_id": match.group().upper()}
            }
    
    # Help/documentation queries
    if any(word in query_lower for word in ['how', 'help', 'password', 'return', 'policy']):
        # Extract search terms
        search_terms = re.sub(r'how (do i|to|can i)', '', query_lower).strip()
        return {
            "tool": "search_knowledge_base",
            "arguments": {"query": search_terms, "limit": 3}
        }
    
    # Shipping queries
    if any(word in query_lower for word in ['shipping', 'ship', 'deliver', 'cost']):
        # Extract weight and destination
        weight_match = re.search(r'(\d+\.?\d*)\s*kg', query_lower)
        dest_match = re.search(r'to\s+(\w+)', query_lower)
        express = 'express' in query_lower
        
        return {
            "tool": "calculate_shipping",
            "arguments": {
                "weight_kg": float(weight_match.group(1)) if weight_match else 1.0,
                "destination": dest_match.group(1) if dest_match else "Unknown",
                "express": express
            }
        }
    
    return None  # No tool needed

# Test queries
test_queries = [
    "What's the status of order ORD-12345?",
    "How do I reset my password?",
    "How much to ship 2.5kg to London?",
    "What time is it?"  # No tool needed
]

print("=== LLM Tool Selection (Simulated) ===")
for query in test_queries:
    result = simulate_llm_tool_selection(query)
    print(f"\nQuery: '{query}'")
    if result:
        print(f"  Tool: {result['tool']}")
        print(f"  Args: {result['arguments']}")
    else:
        print("  No tool selected")

## Part 4: Building the Tool Execution Pipeline

In [ ]:
class ToolExecutor:
    """Safe tool execution with validation."""
    
    def __init__(self, tools: Dict[str, Callable], schemas: list):
        self.tools = tools
        self.schemas = {s['name']: s for s in schemas}
        self.call_history = []
    
    def validate_tool_call(self, tool_name: str, arguments: Dict) -> tuple:
        """Validate tool name and arguments."""
        # Check tool exists
        if tool_name not in self.tools:
            return False, f"Unknown tool: {tool_name}"
        
        # Check required parameters
        schema = self.schemas.get(tool_name)
        if schema:
            required = schema['parameters'].get('required', [])
            for param in required:
                if param not in arguments:
                    return False, f"Missing required parameter: {param}"
        
        return True, "Valid"
    
    def execute(self, tool_name: str, arguments: Dict) -> Dict[str, Any]:
        """Execute a tool with validation."""
        # Validate
        valid, message = self.validate_tool_call(tool_name, arguments)
        if not valid:
            return {"error": message}
        
        # Execute
        try:
            result = self.tools[tool_name](**arguments)
            
            # Log call
            self.call_history.append({
                "tool": tool_name,
                "arguments": arguments,
                "success": True
            })
            
            return result
        except Exception as e:
            return {"error": str(e)}

# Create executor
executor = ToolExecutor(TOOLS, TOOL_SCHEMAS)

# Test execution
print("=== Tool Execution ===")

# Valid call
result = executor.execute("get_order_status", {"order_id": "ORD-12345"})
print("\nValid call result:")
print(json.dumps(result, indent=2))

# Invalid tool
result = executor.execute("delete_everything", {})
print("\nInvalid tool result:")
print(json.dumps(result, indent=2))

# Missing parameter
result = executor.execute("get_order_status", {})
print("\nMissing parameter result:")
print(json.dumps(result, indent=2))

## Part 5: Complete Agent Loop

In [ ]:
def format_response(query: str, tool_result: Optional[Dict]) -> str:
    """
    Simulate LLM formatting the tool result into a response.
    In production, this would be another LLM call with the tool result.
    """
    if tool_result is None:
        return "I don't have a specific tool to help with that. Could you rephrase your question?"
    
    if "error" in tool_result:
        return f"I encountered an issue: {tool_result['error']}. Please check your input and try again."
    
    # Format based on tool type
    if "order_id" in tool_result and "status" in tool_result:
        status = tool_result['status']
        tracking = tool_result.get('tracking_number')
        eta = tool_result.get('estimated_delivery')
        
        response = f"Your order {tool_result['order_id']} is currently {status}."
        if tracking:
            response += f" Tracking number: {tracking}."
        if eta:
            response += f" Expected delivery: {eta}."
        return response
    
    if "articles" in tool_result:
        if not tool_result['articles']:
            return "I couldn't find any relevant articles. Try different search terms."
        articles = tool_result['articles']
        response = f"I found {len(articles)} relevant article(s):\n"
        for i, article in enumerate(articles, 1):
            response += f"\n{i}. **{article['title']}**: {article['content']}"
        return response
    
    if "cost_usd" in tool_result:
        return f"Shipping {tool_result['weight_kg']}kg to {tool_result['destination']} ({tool_result['shipping_type']}): ${tool_result['cost_usd']} ({tool_result['estimated_days']} days)"
    
    return json.dumps(tool_result)

def agent_respond(query: str, executor: ToolExecutor) -> str:
    """Complete agent response pipeline."""
    print(f"\n{'='*50}")
    print(f"User: {query}")
    
    # Step 1: LLM selects tool
    tool_selection = simulate_llm_tool_selection(query)
    
    if tool_selection:
        print(f"\n[Tool Selection] {tool_selection['tool']}")
        print(f"[Arguments] {tool_selection['arguments']}")
        
        # Step 2: Execute tool
        result = executor.execute(tool_selection['tool'], tool_selection['arguments'])
        print(f"[Result] {json.dumps(result)[:100]}...")
    else:
        print("\n[No tool needed]")
        result = None
    
    # Step 3: Format response
    response = format_response(query, result)
    print(f"\nAssistant: {response}")
    
    return response

# Test the agent
queries = [
    "What's the status of order ORD-12345?",
    "How do I reset my password?",
    "What's the shipping cost for 3kg to Paris, express?",
    "Where is order ORD-99999?"  # Non-existent
]

for q in queries:
    agent_respond(q, executor)

## Part 6: Security - Dangerous Operations

In [ ]:
class SecureToolExecutor(ToolExecutor):
    """Tool executor with additional security."""
    
    REQUIRES_CONFIRMATION = ["send_email", "process_refund", "cancel_order"]
    MAX_CALLS_PER_SESSION = 10
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.calls_this_session = 0
    
    def execute(self, tool_name: str, arguments: Dict, user_confirmed: bool = False) -> Dict[str, Any]:
        # Rate limiting
        if self.calls_this_session >= self.MAX_CALLS_PER_SESSION:
            return {"error": "Rate limit exceeded. Please wait before making more requests."}
        
        # Check for dangerous tools
        if tool_name in self.REQUIRES_CONFIRMATION and not user_confirmed:
            return {
                "needs_confirmation": True,
                "action": tool_name,
                "arguments": arguments,
                "message": f"Please confirm: Execute {tool_name} with parameters {arguments}?"
            }
        
        self.calls_this_session += 1
        return super().execute(tool_name, arguments)

# Demonstrate security features
print("=== Security Features ===")

# Add a "dangerous" tool for demonstration
def cancel_order(order_id: str) -> Dict:
    return {"cancelled": True, "order_id": order_id}

TOOLS_WITH_DANGEROUS = {**TOOLS, "cancel_order": cancel_order}

secure_executor = SecureToolExecutor(TOOLS_WITH_DANGEROUS, TOOL_SCHEMAS)

# Try dangerous operation without confirmation
result = secure_executor.execute("cancel_order", {"order_id": "ORD-12345"})
print("\nWithout confirmation:")
print(json.dumps(result, indent=2))

# With confirmation
result = secure_executor.execute("cancel_order", {"order_id": "ORD-12345"}, user_confirmed=True)
print("\nWith confirmation:")
print(json.dumps(result, indent=2))

## Part 7: TODO - Design Your Own Tool

Create a tool for your support chatbot.

In [ ]:
# TODO: Design a new tool schema
# Example: get_product_info, schedule_callback, check_inventory

my_tool_schema = {
    "name": "your_tool_name",
    "description": "Clear description of what this tool does and when to use it",
    "parameters": {
        "type": "object",
        "properties": {
            # Add your parameters here
        },
        "required": []
    }
}

# TODO: Implement the function
def my_tool_function(**kwargs):
    """Your implementation here."""
    pass

print("Design your tool schema and implement the function!")

## Self-Check

Uncomment and run the asserts below to verify your tool calling infrastructure works correctly.

In [ ]:
# SELF-CHECK: Verify your tool calling setup
assert len(TOOL_SCHEMAS) >= 3, "Should define at least 3 tool schemas"
assert all('name' in s and 'parameters' in s for s in TOOL_SCHEMAS), "Each schema needs name and parameters"
assert all(s['name'] in TOOLS for s in TOOL_SCHEMAS), "Each schema must map to an implementation"
assert callable(executor.execute), "ToolExecutor should have execute method"
print(f"✅ Self-check passed! {len(TOOL_SCHEMAS)} tools defined and implemented")

## Part 8: Stakeholder Summary

### TODO: Write a 3-bullet summary (~100 words) for the PM

Template:
• **What tool calling enables:** AI can [look up real-time data, take actions, integrate with systems] instead of relying on training data alone.
• **Safety measures:** We have [validation, confirmation for dangerous ops, rate limiting] to prevent [hallucinated calls, unauthorized actions, abuse].
• **Good candidates:** Tasks that require [current data, external systems, or multi-step workflows] like [order lookup, scheduling, calculations].

### Your Summary:

*Write your explanation here...*

---

## Key Takeaways

1. **Tool calling** lets LLMs interact with external systems
2. **Clear schemas** help the LLM select the right tool
3. **Validation** prevents hallucinated tool calls
4. **Confirmation** for destructive actions
5. **Rate limiting** prevents abuse

### Next Steps
- Explore the interactive playground
- Complete the quiz
- Move to Module 19: Agent Memory